Goal: Calculate necessary data for plotting

Last time everything was combined in the Rdf_to_Figures file making it unnecessarily tedious. This time, we only calculate and save necessary values and will plot it in another notebook.

In addition, the 40Hz FFR which was recorded in both sessions will be merged in section 1 and resampled in section 2. As there was no significant difference between sessions (see Merging_Sessions_Test.notebook and Checking_Merging_Sessions.rmd)



### Note on 04/18/22

THe older version has been modified to address reviewers' comment:

1. The pre-stimulus detrend baseline was changed to -100ms~0ms

2. The noise frequency band has been shortened

3. Drop session 2 and only use session 1 for analyzing the results

In [1]:
cd

/home/luxx0489


In [2]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import sample
import pickle
import mne
from scipy.stats import circmean,circvar
from sklearn import decomposition

import time

# Section 1: Interpreting vetical montage FFR

## Fig. 0 the phase topomap

In [3]:
# first we compare the reference to reveal the difference
# just using PLV and with single channel and all channels

ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
Start_trial_list = [0, 0, 1000]
End_trial_list = [1000, 1000, 2000]
output = np.empty((len(ID_list)*2*len(Freq_list),67))

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# Fz, all chan * reference

row_num = 0

for iSubject in ID_list:# range(2,18):
    
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    # combine and fft
    for iRef in range(2):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data = np.concatenate((data_L,data_H),0)
        
        xw_all = np.fft.rfft(np.hamming(data.shape[2])*data, axis=2)
            
        for iFreq in range(len(Freq_list)):
            
            xw = xw_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]

            plv = np.angle((xw/abs(xw)).mean(0))

            freq = np.fft.rfftfreq(data.shape[2],1/4096)

            index = np.argmin(abs(freq-Freq_list[iFreq]))

            output[row_num,0] = iSubject
            output[row_num,1] = iRef
            output[row_num,2] = Freq_list[iFreq]
            output[row_num,3:67] = plv[:,index]  #Phase of all
            
            row_num +=1

folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF"
file_handle = open(folder_path + '/phase_topomap_all_HL041822.obj','wb')
pickle.dump(output, file_handle)
file_handle.close()

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/luxx0489/.conda/envs/mne/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-758ce52466a6>", line 28, in <module>
    epochs_L.load_data()
  File "/home/luxx0489/.conda/envs/mne/lib/python3.7/site-packages/mne/epochs.py", line 556, in load_data
    self._data = self._get_data()
  File "</home/luxx0489/.conda/envs/mne/lib/python3.7/site-packages/mne/externals/decorator.py:decorator-gen-178>", line 2, in _get_data
  File "/home/luxx0489/.conda/envs/mne/lib/python3.7/site-packages/mne/utils/_logging.py", line 90, in wrapper
    return function(*args, **kwargs)
  File "/home/luxx0489/.conda/envs/mne/lib/python3.7/site-packages/mne/epochs.py", line 1322, in _get_data
    epoch_noproj = self._detrend_offset_decim(epoch_noproj)
  File "</home/luxx0489/.conda/envs/mne/lib/python3.7/site-packages/mne/externals/decorator.py:decorator-ge

KeyboardInterrupt: 

## Fig. 1

In [4]:
folder_path = "./jupyter_projects/MultiChannelSSSR"
file_handle = open(folder_path +'/freq_label.obj','rb')
freq = pickle.load(file_handle)
file_handle.close()

In [5]:
# define functions (requires freq variable)

def ztransform(val, noise):
    return (val-noise.mean())/noise.std()

Target_list = [[34.28,43.43],[91.42,98.28],[233.13,217.13]]
#Noise_range_list = [[20,55],[75,110],[205,240]]

Noise_range_list = [[30,50],[85,105],[215,235]]

Noise_index = []
for iFreq in range(3):
    index1 = np.argmin(abs(freq-Target_list[iFreq][0]))
    index2 = np.argmin(abs(freq-Target_list[iFreq][1]))
    Noise_range = np.arange(np.argmin(abs(freq-Noise_range_list[iFreq][0])),np.argmin(abs(freq-Noise_range_list[iFreq][1])))
    key1 = np.argwhere(Noise_range==index1)
    key2 = np.argwhere(Noise_range==index2)
    
    Noise_index.append(np.delete(Noise_range,[key1,key2]))

In [6]:
# first we compare the reference to reveal the difference
# just using PLV and with single channel and all channels

ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
Start_trial_list = [0, 0, 1000]
End_trial_list = [1000, 1000, 2000]
output = np.empty((len(ID_list)*2*len(Freq_list),7))

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# Fz, all chan * reference

row_num = 0

for iSubject in ID_list:# range(2,18):
    
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    
    # combine and fft
    for iRef in range(2):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:64,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data = np.concatenate((data_L,data_H),0)
        
        xw_all = np.fft.rfft(np.hamming(data.shape[2])*data, axis=2)
            
        for iFreq in range(len(Freq_list)):
            
            xw = xw_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]
            
            data_ave = data[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:].mean(0)
            xw_ave = np.fft.rfft(np.hamming(data_ave.shape[1])*data_ave, axis=1)

            plv = abs((xw/abs(xw)).mean(0))
            mag = abs(xw_ave)
            
            freq = np.fft.rfftfreq(data.shape[2],1/4096)

            index = np.argmin(abs(freq-Freq_list[iFreq]))

            output[row_num,0] = iSubject
            output[row_num,1] = iRef
            output[row_num,2] = Freq_list[iFreq]
            output[row_num,3] = mag[47,index]  #Cz spectral magnitude
            output[row_num,4] = plv[47,index]  #Cz plv
            output[row_num,5] = mag[47,Noise_index[iFreq]].mean() #Fz noise floor
            output[row_num,6] = plv[47,Noise_index[iFreq]].mean() #Cz noise floor
            

            
            row_num +=1
            
file_handle = open(folder_path + '/absolute_PLV_and_ave_Mag_all_HL041822.obj','wb')
pickle.dump(output, file_handle)
file_handle.close()

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub6_HF/Sub6_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3272703  =      0.000 ...   799.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=T

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub12_LF/Sub12_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3321855  =      0.000 ...   811.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub12_HF/Sub12_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3411967  =      0.000 ...   833.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_eve

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub19_LF/Sub19_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3514367  =      0.000 ...   858.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub19_HF/Sub19_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3358719  =      0.000 ...   820.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_eve

## Fig.2 and 3: amplitude and phase of PLV at vertex and mastoids

In [7]:
# first we compare the reference to reveal the difference
# just using PLV and with single channel and all channels

ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
Start_trial_list = [0, 0, 1000, 0, 0, 1000]
End_trial_list = [1000, 1000, 2000, 1000, 1000, 2000]
output = np.empty((len(ID_list)*2*len(Freq_list),11))

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# Fz, all chan * reference

row_num = 0

for iSubject in ID_list:# range(2,18):
    
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    
    # combine and fft
    for iRef in range(1):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data = np.concatenate((data_L,data_H),0)
        linked_mastoid = data[:,64:66,:].mean(1)
        diff_horizontal = data[:,64,:] - data[:,65,:]
        vertex_channel = data[:,47,:] + data[:,46,:]
        refs = np.dstack((linked_mastoid,diff_horizontal,vertex_channel)).transpose(0,2,1)

        xw_all = np.fft.rfft(np.hamming(data.shape[2])*refs, axis=2)
        freq = np.fft.rfftfreq(data.shape[2],1/4096)  
        
        for iFreq in range(len(Freq_list)):
            
            xw = xw_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]

            plv = abs((xw/abs(xw)).mean(0))
            phase = np.angle((xw/abs(xw)).mean(0))


            index = np.argmin(abs(freq-Freq_list[iFreq]))

            output[row_num,0] = iSubject
            output[row_num,1] = Freq_list[iFreq]
            output[row_num,2:5] = plv[:,index]
            output[row_num,5:8] = phase[:,index]
            output[row_num,8:11] = plv[:,Noise_index[iFreq]].mean(1) #Fz noise floor
            
            row_num +=1

file_handle = open(folder_path + '/Fig23_location_PLV_all_HL041822.obj','wb')
pickle.dump(output, file_handle)
file_handle.close()

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub6_HF/Sub6_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3272703  =      0.000 ...   799.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=T

BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3411967  =      0.000 ...   833.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub14_LF/Sub14_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3268607  =      0.000 ...   798.000 secs...
Trigger c

BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3547135  =      0.000 ...   866.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub20_HF/Sub20_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3338239  =      0.000 ...   815.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mea

In [8]:
# first we compare the reference to reveal the difference
# just using PLV and with single channel and all channels

ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
Start_trial_list = [0, 0, 1000, 0, 0, 1000]
End_trial_list = [1000, 1000, 2000, 1000, 1000, 2000]
output = np.empty((len(ID_list)*2*len(Freq_list),11))

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# Fz, all chan * reference

row_num = 0

for iSubject in ID_list:# range(2,18):
    
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    
    # combine and fft
    for iRef in range(1):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data = np.concatenate((data_L,data_H),0)
        linked_mastoid = data[:,64:66,:].mean(1)
        diff_horizontal = data[:,64,:] - data[:,65,:]
        vertex_channel = data[:,47,:] + data[:,46,:]
        refs = np.dstack((linked_mastoid,diff_horizontal,vertex_channel)).transpose(0,2,1)

        freq = np.fft.rfftfreq(data.shape[2],1/4096)  
        
        for iFreq in range(len(Freq_list)):
            
            refs_ave = refs[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:].mean(0)
            xw_ave = np.fft.rfft(np.hamming(refs_ave.shape[1])*refs_ave, axis=1)
            
            mag = abs(xw_ave)

            index = np.argmin(abs(freq-Freq_list[iFreq]))

            output[row_num,0] = iSubject
            output[row_num,1] = Freq_list[iFreq]
            output[row_num,2:5] = mag[:,index]
            output[row_num,5:8] = 0
            output[row_num,8:11] = mag[:,Noise_index[iFreq]].mean(1) #Fz noise floor
            
            row_num +=1

file_handle = open(folder_path + '/Fig23_location_mag_all_HL041822.obj','wb')
pickle.dump(output, file_handle)
file_handle.close()

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub6_HF/Sub6_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3272703  =      0.000 ...   799.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=T

BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3411967  =      0.000 ...   833.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub14_LF/Sub14_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3268607  =      0.000 ...   798.000 secs...
Trigger c

BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3547135  =      0.000 ...   866.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub20_HF/Sub20_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3338239  =      0.000 ...   815.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mea

# Section 2. Compare multichannel recording methods

In [8]:
# define functions (requires freq variable)

freq = np.fft.rfftfreq(1792,1/4096)

def ztransform(val, noise):
    return (val-noise.mean())/noise.std()

Target_list = [[34.28,43.43],[91.42,98.28],[233.13,217.13]]
#Noise_range_list = [[20,55],[75,110],[205,240]]
Noise_range_list = [[30,50],[85,105],[215,235]]

Noise_index = []
for iFreq in range(3):
    index1 = np.argmin(abs(freq-Target_list[iFreq][0]))
    index2 = np.argmin(abs(freq-Target_list[iFreq][1]))
    Noise_range = np.arange(np.argmin(abs(freq-Noise_range_list[iFreq][0])),np.argmin(abs(freq-Noise_range_list[iFreq][1])))
    key1 = np.argwhere(Noise_range==index1)
    key2 = np.argwhere(Noise_range==index2)
    
    Noise_index.append(np.delete(Noise_range,[key1,key2]))
    


In [4]:
ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
sample_size = [200,400,600,800,1000]
output = np.empty((len(ID_list)*2*len(Freq_list)*len(sample_size),16))
Start_trial_list = [0, 0, 1000]
End_trial_list = [1000, 1000, 2000]

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# all chan plv, 
# all chan ITC,
# all chan T2,
# tPCA, cPCA

row_num = 0

for iSubject in ID_list:
    
    t = time.time()
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    
    print('loading took %s' % (time.time()-t))
    t = time.time()
    
    # combine and fft
    for iRef in range(2):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data_all = np.concatenate((data_L,data_H),0)
        xw_all = np.fft.rfft(np.hamming(data.shape[2])*data_all, axis=2)
        
        for iFreq in range(len(Freq_list)):
            
            xw_session = xw_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]
            data = data_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]
            
            for iSize in range(len(sample_size)):
                ind_sample = sample(range(End_trial_list[iFreq]- Start_trial_list[iFreq]),sample_size[iSize])
                xw=xw_session[ind_sample,:,:]
                
                # plv and itc
                plv = abs((xw/abs(xw)).mean(0))
                itc = abs(xw.mean(0))/(abs(xw).mean(0))
                
                # plv and itc, mean and rms
                plv_mean = plv[0:64,:].mean(0)
                itc_mean = itc[0:64,:].mean(0)
                plv_rms = np.sqrt((plv[0:64,:]**2).mean(0))
                itc_rms = np.sqrt((itc[0:64,:]**2).mean(0))
                
                print('PLV took %s' % (time.time()-t))
                t = time.time()
                
                # hotelling's t2
                out = xw[:,0:64,:]
                sample_dat = np.concatenate((np.real(out),np.imag(out)),axis=1)
                T2 = np.zeros(sample_dat.shape[2])
                
                for iBar in range(5,sample_dat.shape[2]-3):
                    batch = sample_dat[:,:,iBar].transpose(1,0)
                    S_inv = np.linalg.inv(np.cov(batch))
                    T2[iBar]=batch.mean(1) @S_inv @batch.mean(1)
                
                print('T2 took %s' % (time.time()-t))
                t = time.time()
                
                
                #cPCA and MMSC, share a loop because they both use csd
                plv_mat=np.empty((64,xw.shape[2]))
                msmc = np.empty(xw.shape[2])
                for iBar in range(max(Noise_index[2])+1):   
                    #as this section is most computationally expensive, not all bars will be calculated
                    # only those frequency bars that will be used in calculating normalized scores are used
                    # i.e. max(Noize_index)
                    
                    # cPCA
                    csd = np.matmul(xw[:,0:64,iBar].T,xw[:,0:64,iBar].conj())
                    w,v = np.linalg.eig(csd)
                    pc = np.matmul(v.conj().T,xw[:,0:64,iBar].T)
                    cplv = (pc/abs(pc)).mean(1)
                    plv_mat[:,iBar] = abs(cplv)
                
                    # MSMC
                    v_m = np.matrix(xw[:,0:64,iBar].mean(0))
                    msmc[iBar] = np.real((v_m.conj() @ np.linalg.inv(csd) @ v_m.T)[0,0])
                
                
                print('cPCA and MMSC took %s' % (time.time()-t))
                t = time.time()
                    
                    
                # concatenate and tPCA
                mix_long = np.concatenate([data[iTrial,0:64,:] for iTrial in ind_sample], axis=1)

                pca = decomposition.PCA(n_components=1)
                pca.fit(mix_long.T)
                pc_t = np.empty((sample_size[iSize],data.shape[2]))
                for iTrial in range(len(ind_sample)):
                    pc_t[iTrial,:] = pca.transform(data[ind_sample[iTrial],0:64,:].T).squeeze()
                xw_pc_t = np.fft.rfft(pc_t, axis=1)
                xw_pc_t = xw_pc_t/abs(xw_pc_t)
                plv_pc_t = abs(xw_pc_t.mean(0))
                
                print('tPCA took %s' % (time.time()-t))
                t = time.time()
                
                index = np.argmin(abs(freq-Freq_list[iFreq]))

                output[row_num,0] = iSubject
                output[row_num,1] = iRef
                output[row_num,2] = Freq_list[iFreq]
                output[row_num,3] = sample_size[iSize]
                output[row_num,4] = ztransform(plv[37,index],plv[37,Noise_index[iFreq]])
                output[row_num,5] = ztransform(itc[37,index],itc[37,Noise_index[iFreq]])
                output[row_num,6] = ztransform(plv[47,index],plv[47,Noise_index[iFreq]])
                output[row_num,7] = ztransform(itc[47,index],itc[47,Noise_index[iFreq]])
                output[row_num,8] = ztransform(plv_mean[index],plv_mean[Noise_index[iFreq]])
                output[row_num,9] = ztransform(itc_mean[index],itc_mean[Noise_index[iFreq]])
                output[row_num,10] = ztransform(plv_rms[index],plv_mean[Noise_index[iFreq]])
                output[row_num,11] = ztransform(itc_rms[index],itc_mean[Noise_index[iFreq]])
                output[row_num,12] = ztransform(T2[index],T2[Noise_index[iFreq]])
                output[row_num,13] = ztransform(plv_mat[0,index],plv_mat[0,Noise_index[iFreq]])
                output[row_num,14] = ztransform(plv_pc_t[index],plv_pc_t[Noise_index[iFreq]])
                output[row_num,15] = ztransform(msmc[index], msmc[Noise_index[iFreq]])

                row_num +=1

file_handle = open(folder_path + '/method_comparison_all_HL041822.obj','wb')
pickle.dump(output, file_handle)
file_handle.close()

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub6_HF/Sub6_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3272703  =      0.000 ...   799.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=T

NameError: name 'data' is not defined

The following code is added to provide the absolute measurement for the reviewers

In [11]:
ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
sample_size = [200,400,600,800,1000]

# the original output matrix was used to stroe the absolute values, and the new one is for the noise floor
output = np.empty((len(ID_list)*2*len(Freq_list)*len(sample_size),16))
output_noise = np.empty((len(ID_list)*2*len(Freq_list)*len(sample_size),16))
output_noise_std = np.empty((len(ID_list)*2*len(Freq_list)*len(sample_size),16))

Start_trial_list = [0, 0, 1000]
End_trial_list = [1000, 1000, 2000]

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# all chan plv, 
# all chan ITC,
# all chan T2,
# tPCA, cPCA

row_num = 0

for iSubject in ID_list:
    
    t = time.time()
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    
    print('loading took %s' % (time.time()-t))
    t = time.time()
    
    # combine and fft
    for iRef in range(2):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data_all = np.concatenate((data_L,data_H),0)
        xw_all = np.fft.rfft(np.hamming(data_all.shape[2])*data_all, axis=2)
        
        
        for iFreq in range(len(Freq_list)):
            
            xw_session = xw_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]
            data = data_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]
            
            for iSize in range(len(sample_size)):
                ind_sample = sample(range(End_trial_list[iFreq]- Start_trial_list[iFreq]),sample_size[iSize])
                xw=xw_session[ind_sample,:,:]
                
                # plv and itc
                plv = abs((xw/abs(xw)).mean(0))
                itc = abs(xw.mean(0))/(abs(xw).mean(0))
                
                # plv and itc, mean and rms
                plv_mean = plv[0:64,:].mean(0)
                itc_mean = itc[0:64,:].mean(0)
                plv_rms = np.sqrt((plv[0:64,:]**2).mean(0))
                itc_rms = np.sqrt((itc[0:64,:]**2).mean(0))
                
                print('PLV took %s' % (time.time()-t))
                t = time.time()
                
                # hotelling's t2
                out = xw[:,0:64,:]
                sample_dat = np.concatenate((np.real(out),np.imag(out)),axis=1)
                T2 = np.zeros(sample_dat.shape[2])
                
                for iBar in range(5,sample_dat.shape[2]-3):
                    batch = sample_dat[:,:,iBar].transpose(1,0)
                    S_inv = np.linalg.inv(np.cov(batch))
                    T2[iBar]=batch.mean(1) @S_inv @batch.mean(1)
                
                print('T2 took %s' % (time.time()-t))
                t = time.time()
                
                
                #cPCA and MMSC, share a loop because they both use csd
                plv_mat=np.empty((64,xw.shape[2]))
                msmc = np.empty(xw.shape[2])
                for iBar in range(max(Noise_index[2])+1):   
                    #as this section is most computationally expensive, not all bars will be calculated
                    # only those frequency bars that will be used in calculating normalized scores are used
                    # i.e. max(Noize_index)
                    
                    # cPCA
                    csd = np.matmul(xw[:,0:64,iBar].T,xw[:,0:64,iBar].conj())
                    w,v = np.linalg.eig(csd)
                    pc = np.matmul(v.conj().T,xw[:,0:64,iBar].T)
                    cplv = (pc/abs(pc)).mean(1)
                    plv_mat[:,iBar] = abs(cplv)
                
                    # MSMC
                    v_m = np.matrix(xw[:,0:64,iBar].mean(0))
                    msmc[iBar] = np.real((v_m.conj() @ np.linalg.inv(csd) @ v_m.T)[0,0])
                
                
                print('cPCA and MMSC took %s' % (time.time()-t))
                t = time.time()
                    
                    
                # concatenate and tPCA
                mix_long = np.concatenate([data[iTrial,0:64,:] for iTrial in ind_sample], axis=1)

                pca = decomposition.PCA(n_components=1)
                pca.fit(mix_long.T)
                pc_t = np.empty((sample_size[iSize],data.shape[2]))
                for iTrial in range(len(ind_sample)):
                    pc_t[iTrial,:] = pca.transform(data[ind_sample[iTrial],0:64,:].T).squeeze()
                xw_pc_t = np.fft.rfft(pc_t, axis=1)
                xw_pc_t = xw_pc_t/abs(xw_pc_t)
                plv_pc_t = abs(xw_pc_t.mean(0))
                
                print('tPCA took %s' % (time.time()-t))
                t = time.time()
                
                index = np.argmin(abs(freq-Freq_list[iFreq]))

                output[row_num,0] = iSubject
                output[row_num,1] = iRef
                output[row_num,2] = Freq_list[iFreq]
                output[row_num,3] = sample_size[iSize]
                output[row_num,4] = plv[37,index]
                output[row_num,5] = itc[37,index]
                output[row_num,6] = plv[47,index]
                output[row_num,7] = itc[47,index]
                output[row_num,8] = plv_mean[index]
                output[row_num,9] = itc_mean[index]
                output[row_num,10] = plv_rms[index]
                output[row_num,11] = itc_rms[index]
                output[row_num,12] = T2[index]
                output[row_num,13] = plv_mat[0,index]
                output[row_num,14] = plv_pc_t[index]
                output[row_num,15] = msmc[index]
                
                output_noise[row_num,0] = iSubject
                output_noise[row_num,1] = iRef
                output_noise[row_num,2] = Freq_list[iFreq]
                output_noise[row_num,3] = sample_size[iSize]
                output_noise[row_num,4] = plv[37,Noise_index[iFreq]].mean()
                output_noise[row_num,5] = itc[37,Noise_index[iFreq]].mean()
                output_noise[row_num,6] = plv[47,Noise_index[iFreq]].mean()
                output_noise[row_num,7] = itc[47,Noise_index[iFreq]].mean()
                output_noise[row_num,8] = plv_mean[Noise_index[iFreq]].mean()
                output_noise[row_num,9] = itc_mean[Noise_index[iFreq]].mean()
                output_noise[row_num,10] = plv_mean[Noise_index[iFreq]].mean()
                output_noise[row_num,11] = itc_mean[Noise_index[iFreq]].mean()
                output_noise[row_num,12] = T2[Noise_index[iFreq]].mean()
                output_noise[row_num,13] = plv_mat[0,Noise_index[iFreq]].mean()
                output_noise[row_num,14] = plv_pc_t[Noise_index[iFreq]].mean()
                output_noise[row_num,15] = msmc[Noise_index[iFreq]].mean()
                
                output_noise_std[row_num,0] = iSubject
                output_noise_std[row_num,1] = iRef
                output_noise_std[row_num,2] = Freq_list[iFreq]
                output_noise_std[row_num,3] = sample_size[iSize]
                output_noise_std[row_num,4] = plv[37,Noise_index[iFreq]].std()
                output_noise_std[row_num,5] = itc[37,Noise_index[iFreq]].std()
                output_noise_std[row_num,6] = plv[47,Noise_index[iFreq]].std()
                output_noise_std[row_num,7] = itc[47,Noise_index[iFreq]].std()
                output_noise_std[row_num,8] = plv_mean[Noise_index[iFreq]].std()
                output_noise_std[row_num,9] = itc_mean[Noise_index[iFreq]].std()
                output_noise_std[row_num,10] = plv_mean[Noise_index[iFreq]].std()
                output_noise_std[row_num,11] = itc_mean[Noise_index[iFreq]].std()
                output_noise_std[row_num,12] = T2[Noise_index[iFreq]].std()
                output_noise_std[row_num,13] = plv_mat[0,Noise_index[iFreq]].std()
                output_noise_std[row_num,14] = plv_pc_t[Noise_index[iFreq]].std()
                output_noise_std[row_num,15] = msmc[Noise_index[iFreq]].std()

                row_num +=1

file_handle = open(folder_path + '/method_comparison_all_abs_w_std_HL041822.obj','wb')
pickle.dump(output, file_handle)
pickle.dump(output_noise, file_handle)
pickle.dump(output_noise_std, file_handle)

file_handle.close()

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub6_HF/Sub6_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3272703  =      0.000 ...   799.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=T

tPCA took 2.506054401397705
PLV took 2.248945713043213
T2 took 1.9760785102844238
cPCA and MMSC took 4.039797782897949
tPCA took 3.8131678104400635
PLV took 3.0179171562194824
T2 took 2.431678533554077
cPCA and MMSC took 5.238464832305908
tPCA took 5.325263261795044
PLV took 3.7551445960998535
T2 took 2.8740007877349854
cPCA and MMSC took 6.41114616394043
tPCA took 6.634815216064453
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
PLV took 4.2215352058410645
T2 took 1.106931209564209
cPCA and MMSC took 1.5176749229431152
tPCA took 1.13496732711792
PLV took 1.5203588008880615
T2 took 1.5390195846557617
cPCA and MMSC took 2.8519840240478516
tPCA took 2.4891650676727295
PLV took 2.2635412216186523
T2 took 1.9744198322296143
cPCA and MMSC took 4.069429636001587
tPCA took 3.8250226974487305
PLV took 2.9909613132476807
T2 took 2.427313804626465
cPCA and MMSC took 5.237503290176392
tPCA 

0 bad epochs dropped
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_HF/Sub9_HF/Sub9_HF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3305471  =      0.000 ...   807.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
loading took 11.94803237915039
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
PLV took 5.535442590713501
T2 took 1.1139416694641113
cPCA and MMSC took 1.5123343467712402
tPCA too

tPCA took 1.2710044384002686
PLV took 1.5212922096252441
T2 took 1.5358192920684814
cPCA and MMSC took 2.8660874366760254
tPCA took 2.775731325149536
PLV took 2.5199718475341797
T2 took 2.0960135459899902
cPCA and MMSC took 4.207089424133301
tPCA took 4.145435094833374
PLV took 3.3381738662719727
T2 took 2.494025230407715
cPCA and MMSC took 5.422681570053101
tPCA took 5.4272987842559814
PLV took 4.166382074356079
T2 took 2.960770606994629
cPCA and MMSC took 6.639861822128296
tPCA took 6.8358471393585205
PLV took 0.8798613548278809
T2 took 1.2706358432769775
cPCA and MMSC took 1.646526575088501
tPCA took 1.273993730545044
PLV took 1.5268266201019287
T2 took 1.5365934371948242
cPCA and MMSC took 2.8913445472717285
tPCA took 2.6521904468536377
PLV took 2.252641439437866
T2 took 1.9876434803009033
cPCA and MMSC took 4.112907648086548
tPCA took 3.915653944015503
PLV took 2.989063262939453
T2 took 2.4227421283721924
cPCA and MMSC took 5.242043972015381
tPCA took 5.344399690628052
PLV took 3.

0 bad epochs dropped
loading took 13.207707643508911
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
PLV took 5.493927717208862
T2 took 1.0974092483520508
cPCA and MMSC took 1.5069282054901123
tPCA took 1.2614820003509521
PLV took 1.5213890075683594
T2 took 1.523423194885254
cPCA and MMSC took 2.8704607486724854
tPCA took 2.578244209289551
PLV took 2.248361110687256
T2 took 1.9839065074920654
cPCA and MMSC took 4.04381799697876
tPCA took 4.566142797470093
PLV took 2.9867091178894043
T2 took 2.4133059978485107
cPCA and MMSC took 5.201866149902344
tPCA took 5.30673885345459
PLV took 3.729196548461914
T2 took 2.8554773330688477
cPCA and MMSC took 6.409918785095215
tPCA took 6.639507055282593
PLV took 0.7861714363098145
T2 took 1.108088493347168
cPCA and MMSC took 1.51609468460083
tPCA took 1.235245943069458
PLV took 1.522010326

PLV took 1.5782294273376465
T2 took 1.5408306121826172
cPCA and MMSC took 2.9422881603240967
tPCA took 2.6832504272460938
PLV took 2.3760061264038086
T2 took 1.993835210800171
cPCA and MMSC took 4.326953887939453
tPCA took 4.011054992675781
PLV took 3.161019802093506
T2 took 2.5657219886779785
cPCA and MMSC took 5.624671459197998
tPCA took 6.492570161819458
PLV took 3.9541711807250977
T2 took 3.5423083305358887
cPCA and MMSC took 6.939358234405518
tPCA took 7.395519495010376
PLV took 0.8323290348052979
T2 took 1.1241421699523926
cPCA and MMSC took 1.5900535583496094
tPCA took 1.3509433269500732
PLV took 1.616945505142212
T2 took 1.5848362445831299
cPCA and MMSC took 3.0338895320892334
tPCA took 2.71901798248291
PLV took 2.3768770694732666
T2 took 2.152316093444824
cPCA and MMSC took 4.3304407596588135
tPCA took 4.207789897918701
PLV took 3.1605746746063232
T2 took 2.683486223220825
cPCA and MMSC took 5.626344203948975
tPCA took 5.475949287414551
PLV took 3.9479923248291016
T2 took 3.41

PLV took 3.338240385055542
T2 took 3.4438157081604004
cPCA and MMSC took 5.901977300643921
tPCA took 6.071599960327148
PLV took 4.172452449798584
T2 took 4.559723377227783
cPCA and MMSC took 7.235422611236572
tPCA took 7.5744476318359375
PLV took 0.8723139762878418
T2 took 1.1437928676605225
cPCA and MMSC took 1.6605639457702637
tPCA took 1.4291608333587646
PLV took 1.7008283138275146
T2 took 1.7310330867767334
cPCA and MMSC took 3.178476572036743
tPCA took 2.9559741020202637
PLV took 2.514651298522949
T2 took 2.4395716190338135
cPCA and MMSC took 4.487800359725952
tPCA took 4.518013954162598
PLV took 3.352663278579712
T2 took 3.472383975982666
cPCA and MMSC took 5.798712730407715
tPCA took 6.128978967666626
PLV took 4.185834646224976
T2 took 4.959468603134155
cPCA and MMSC took 7.312435626983643
tPCA took 7.698141813278198
PLV took 0.8723580837249756
T2 took 1.1518397331237793
cPCA and MMSC took 1.670039415359497
tPCA took 1.4312374591827393
PLV took 1.6991569995880127
T2 took 1.68500

cPCA and MMSC took 3.1583425998687744
tPCA took 2.9708399772644043
PLV took 2.520035982131958
T2 took 2.54862904548645
cPCA and MMSC took 4.485779523849487
tPCA took 4.487025022506714
PLV took 3.350792646408081
T2 took 3.5077767372131348
cPCA and MMSC took 5.928167819976807
tPCA took 6.273850202560425
PLV took 4.181206464767456
T2 took 5.207131624221802
cPCA and MMSC took 7.311870336532593
tPCA took 7.541177272796631
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub20_LF/Sub20_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3547135  =      0.000 ...   866.000 secs...
Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 eve

cPCA and MMSC took 5.786174297332764
tPCA took 5.892458200454712
PLV took 4.186538219451904
T2 took 4.681840896606445
cPCA and MMSC took 7.360514163970947
tPCA took 7.421316862106323
PLV took 0.8736433982849121
T2 took 1.1748335361480713
cPCA and MMSC took 1.6319406032562256
tPCA took 1.4232735633850098
PLV took 1.6970388889312744
T2 took 1.7705235481262207
cPCA and MMSC took 3.1441245079040527
tPCA took 2.891467332839966
PLV took 2.520778179168701
T2 took 2.5094172954559326
cPCA and MMSC took 4.496935844421387
tPCA took 4.466961860656738
PLV took 3.3539700508117676
T2 took 3.5776405334472656
cPCA and MMSC took 5.909717321395874
tPCA took 6.027724742889404
PLV took 4.185148239135742
T2 took 4.9071221351623535
cPCA and MMSC took 7.540510416030884
tPCA took 7.681178092956543
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
PLV took 5.25849723815918
T2 took 1.1811394691467285
cPCA an

TypeError: dump() missing required argument 'file' (pos 2)

The following code is for analyzing the temporal response

In [68]:
ID_list = [6,7,8,9,10,12,14,16,17,18,19,20,21,22]
Freq_list = [34.28, 91.42, 217.13]
sample_size = [200,400,600,800,1000]
output = np.empty((len(ID_list)*2*len(Freq_list)*len(sample_size),16))
Start_trial_list = [0, 0, 1000]
End_trial_list = [1000, 1000, 2000]

# ID, reference(0 for ave, 1 for mastoid), frequency(43.43 and 98),
# all chan plv, 
# all chan ITC,
# all chan T2,
# tPCA, cPCA

temporal_mat = np.zeros((len(ID_list), 2, len(Freq_list), 2, int((0.43752)*4096)))

row_num = 0

for iSubject in ID_list:
    
    t = time.time()
    #load both sessions
    for file_name in ['LF','HF']:

        folder_path = "./jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_"+ file_name +"/"
        sub_id = 'Sub' + str(iSubject) + '_' + file_name
        raw = mne.io.read_raw_bdf(folder_path + sub_id + '/' + sub_id + '_TMR0_MSK.bdf',
                                       stim_channel='Status', preload=True, exclude = ['EXG3','EXG4','EXG5','EXG6','EXG7','EXG8'])
        
        raw.filter(20, 300., fir_design='firwin')
        events = mne.find_events(raw)
        
        if file_name == 'LF':
            epochs_L = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_L.load_data()
        else:
            epochs_H = mne.Epochs(raw,events=events, tmin=-0.1, tmax=0.5)
            epochs_H.load_data()
    
    print('loading took %s' % (time.time()-t))
    t = time.time()
    
    # combine and fft
    for iRef in range(2):
        if iRef==0:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels='average')\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        else:
            data_L = epochs_L.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            data_H = epochs_H.copy().set_eeg_reference(ref_channels=['EXG1','EXG2'])\
            .get_data()[:,0:67,int(np.round((0.1)*4096)):int(np.round((0.1+0.43752)*4096))]
            
        data_all = np.concatenate((data_L,data_H),0)
        
        for iFreq in range(len(Freq_list)):
            
            data = data_all[Start_trial_list[iFreq]:End_trial_list[iFreq],:,:]
            data_ave = data.mean(0)
            
            temporal_mat[ID_list.index(iSubject), iRef, iFreq, 0, :] = data_ave[47,:]
            temporal_mat[ID_list.index(iSubject), iRef, iFreq, 1, :] = data_ave[64:66,:].mean(0)

Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub6_LF/Sub6_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3284991  =      0.000 ...   802.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 20 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 20.00
- Lower transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 17.50 Hz)
- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutoff frequency: 337.50 Hz)
- Filter length: 2703 samples (0.660 sec)

Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65

Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub9_LF/Sub9_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3268607  =      0.000 ...   798.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 20 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 20.00
- Lower transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 17.50 Hz)
- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutof

0 bad epochs dropped
loading took 18.68265128135681
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub14_LF/Sub14_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3268607  =      0.000 ...   798.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 20 - 3e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB st

1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
loading took 19.251755952835083
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40_100_200Hz_LF/Sub18_LF/Sub18_LF_TMR0_MSK.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3313663  =      0.000 ...   809.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter fro

- Upper passband edge: 300.00 Hz
- Upper transition bandwidth: 75.00 Hz (-6 dB cutoff frequency: 337.50 Hz)
- Filter length: 2703 samples (0.660 sec)

Trigger channel has a non-zero initial value of 130944 (consider using initial_event=True to detect this event)
1000 events found
Event IDs: [65409 65410]
1000 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1000 events and 2459 original time points ...
0 bad epochs dropped
loading took 19.764342308044434
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Extracting EDF parameters from /home/luxx0489/jupyter_projects/MultiChannelSSSR/EEG_data_40